In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.preprocessing import sequence
from keras.preprocessing import text
from keras.layers import Conv1D, Flatten,MaxPooling1D
import pandas as pd

from sklearn.model_selection import KFold

# Get files from Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path_to_file = '/content/drive/My Drive/Colab Notebooks/train-balanced-sarcasm.csv'
sarcasm_df = pd.read_csv(path_to_file)

Mounted at /content/drive


In [3]:
path_to_file = '/content/drive/My Drive/Colab Notebooks/test-balanced-final.csv'
test = pd.read_csv(path_to_file)


In [4]:
!pip install transformers
from transformers import DistilBertTokenizerFast

# Instantiate DistilBERT tokenizer...we use the Fast version to optimize runtime
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.0 MB/s 
     |████████████████████████████████| 182 kB 53.5 MB/s 
     |████████████████████████████████| 7.6 MB 57.5 MB/s 


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
!pip install datasets
import datasets
from datasets import Dataset, DatasetDict

batch_2 = sarcasm_df.sample(frac=1/400, random_state=4).reset_index(drop=True)

list=[]
for i in range(0,len(batch_2)):
  if (type(batch_2['comment'][i]) == float):
    list += [i]
  elif (type(batch_2['comment'][i]) != float) and (len(tokenizer.encode(batch_2['comment'][i], add_special_tokens=True))>512):
    list += [i]
batch_2.drop(batch_2.index[list],inplace=True)
batch_2 = batch_2[["label", "comment"]]


list=[]
for i in range(0,len(test)):
  if (type(test['comment'][i]) == float):
    list += [i]
  elif (type(test['comment'][i]) != float) and (len(tokenizer.encode(test['comment'][i], add_special_tokens=True))>512):
    list += [i]
test.drop(test.index[list],inplace=True)
test = test[["label", "comment"]]






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 66.1 MB/s 
     |████████████████████████████████| 115 kB 72.5 MB/s 
     |████████████████████████████████| 127 kB 58.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Token indices sequence length is longer than the specified maximum sequence length for this model (1038 > 512). Running this sequence through the model will result in indexing errors


In [6]:
#first 2000 is train
train = batch_2[:2000]
#next 500 is validation
val = batch_2[2000:]

In [7]:
ds_train = Dataset.from_pandas(train)
ds_train = ds_train.rename_column("comment", "text")
ds_val = Dataset.from_pandas(val)
ds_val = ds_val.rename_column("comment", "text")
test = Dataset.from_pandas(test)
test = test.rename_column("comment", "text")

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


token_train = ds_train.map(tokenize_function, batched=True)
token_val = ds_val.map(tokenize_function, batched=True)
token_test = test.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/252 [00:00<?, ?ba/s]

In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer2")

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5, num_hidden_layers=15)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [11]:
!pip install evaluate
!pip install bert_score
import numpy as np
import evaluate
metric = evaluate.combine(['f1','accuracy','precision','recall'])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 888 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 3.5 MB/s 


In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=token_train,
    eval_dataset=token_val,
    compute_metrics=compute_metrics,
)

In [ ]:
#repeat with different number of hidden layers to choose most successful model
trainer.train()

In [ ]:
trainer.predict(token_test)